In [1]:
import numpy as np
import import_ipynb
#import numpy.linalg as LA
#import LMM as lmm
import random
import time
import os
import pandas as pd
from scipy.io import mmread
import scipy.sparse as sp
import matplotlib.pyplot as plt
from IPython.display import Image
import scanpy as sc
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import matplotlib.patches as mpatches
from statsmodels.graphics.api import abline_plot
import seaborn as sns
import umap
from numpy import eye, asarray, dot, sum, diag
from numpy.linalg import svd

import statsmodels.formula.api as smf
import pandas


np.random.seed(10)

def get_binary_covariate(covariate, covariate_level, data):
    covariate_list = np.zeros((data.obs.shape[0]))
    for i in range(data.obs.shape[0]):
        ### select the ith element of 
        if data.obs[[covariate]].squeeze()[i] == covariate_level:
            covariate_list[i] = 1
    return covariate_list




def varimax(Phi, gamma = 1.0, q = 100, tol = 1e-6):
    p,k = Phi.shape
    R = eye(k)
    d=0
    for i in range(q):
        d_old = d
        Lambda = dot(Phi, R)
        u,s,vh = svd(dot(Phi.T,asarray(Lambda)**3 - (gamma/p) * dot(Lambda, diag(diag(dot(Lambda.T,Lambda))))))
        R = dot(u,vh)
        d = sum(s)
        if d_old!=0 and d/d_old < 1 + tol: break
    Lambda = dot(Phi, R)

    return {'rotloadings':Lambda, 'rotmat':R}



In [2]:
#### import the immune subpopulation of the rat samples
data = sc.read('/home/delaram/scLMM/input_data_designMat/inputdata_rat_set1_countData_2.h5ad') ## attributes removed
data.var_names_make_unique()
# a.obs['orig.ident'].head()
### renaming the meta info column names: https://github.com/theislab/scvelo/issues/255
data.__dict__['_raw'].__dict__['_var'] = data.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})

data_numpy = data.X.toarray()
cell_sums = np.sum(data_numpy,axis=1) # row sums - library size
gene_sums = np.sum(data_numpy,axis=0) # col sums - sum reads in a gene
gene_vars = np.var(data_numpy, axis=0)
data_numpy = data_numpy[:,gene_sums != 0]

### print head pf the data.obs
print(data.obs.head())
data_sub = data_numpy
strain = data.obs.strain
#### sample metadata
y_cluster =data.obs.cluster 
y_sample = data.obs.sample
y_sample = data.obs[['sample']].squeeze()
y_strain_str = data.obs[['strain']].squeeze()

y_strain = get_binary_covariate('strain', 'DA', data=data)


## working with the rat data
num_cells = data_sub.shape[0]
num_genes = data_sub.shape[1]
num_genes = 5000

#### select num_genes genes based on variance
### calculate the variance for each gene
gene_vars = np.var(data_numpy, axis=0)
### select the top num_genes genes with the highest variance
gene_idx = np.argsort(gene_vars)[::-1][0:num_genes]
### subset the data matrix to the top num_genes genes
data_numpy = data_numpy[:, gene_idx]

#### randomly select num_genes genes
#gene_idx = random.sample(range(0, data_numpy.shape[1]), num_genes)
#data_numpy = data_numpy[:, gene_idx]

y = data_numpy


                          orig.ident  nCount_RNA  nFeature_RNA sample strain  \
DA_01_AAACCTGAGACTTGAA-1           0      3007.0          1244  DA_01     DA   
DA_01_AAACCTGAGCCATCGC-1           0      2176.0           919  DA_01     DA   
DA_01_AAACCTGAGGGAAACA-1           0      1978.0           829  DA_01     DA   
DA_01_AAACCTGCACCCATTC-1           0      2702.0          1104  DA_01     DA   
DA_01_AAACCTGCATGCATGT-1           0      2664.0          1061  DA_01     DA   

                         cluster           refined_cell_ID  
DA_01_AAACCTGAGACTTGAA-1       5  DA_01_AAACCTGAGACTTGAA-1  
DA_01_AAACCTGAGCCATCGC-1       9  DA_01_AAACCTGAGCCATCGC-1  
DA_01_AAACCTGAGGGAAACA-1       2  DA_01_AAACCTGAGGGAAACA-1  
DA_01_AAACCTGCACCCATTC-1      11  DA_01_AAACCTGCACCCATTC-1  
DA_01_AAACCTGCATGCATGT-1       5  DA_01_AAACCTGCATGCATGT-1  


In [ ]:
### generating the binary sample info for design matrix
y_DA_01 = get_binary_covariate('sample', 'DA_01', data=data)
y_DA_02 = get_binary_covariate('sample', 'DA_02', data=data)
y_LEW_01 = get_binary_covariate('sample', 'LEW_01', data=data)
y_LEW_02 = get_binary_covariate('sample', 'LEW_02', data=data)

In [6]:
# create a one hot encoded array of the y_cluster
y_cluster_int = np.array(y_cluster, dtype=int) ## convert the y_cluster to a numpy array of integers
encoded_array = np.zeros((y_cluster_int.size, y_cluster_int.max()+1), dtype=int) #creating a 2D array filled with 0's
encoded_array[np.arange(y_cluster_int.size),y_cluster_int] = 1  #replacing 0 with a 1 at the index of the original array

## calculate the column sums of the encoded array
col_sums = np.sum(encoded_array, axis=0)
## get the number of unique clusters in y_cluster
num_clusters = len(np.unique(y_cluster))

print(num_clusters)
print(len(col_sums)) ## should be equal to the number of unique clusters
print(col_sums)

## remove the last column of the encoded array - x needs to be full rank
encoded_array = encoded_array[:,0:num_clusters-1] # why isn't this needed????

17
17
[4989 4212 2606 2091 1740 1150 1124  984  977  713  518  517  446  410
  265  167  127]


In [ ]:
#### Design matrix : Intercept + Depth
x = np.column_stack((np.ones((data.obs.shape[0],1)), data.obs.nCount_RNA))
print(x[0:3,:])
print(x.shape)

In [ ]:
#### Design matrix : Intercept + sample
x = np.column_stack((y_DA_01, y_DA_02, y_LEW_01, y_LEW_02))
x = sm.add_constant(x) ## adding the intercept
print(x[0:3,:])
print(x.shape)

In [ ]:
#### Design matrix : Intercept + Depth + sample
x = np.column_stack((y_DA_01, y_DA_02, y_LEW_01, y_LEW_02))
x = np.column_stack((x, data.obs.nCount_RNA))
x = sm.add_constant(x) ## adding the intercept

print(x[0:3,:])
print(x.shape)

In [ ]:
#### Design matrix : Intercept + Depth + cell-types
x = np.column_stack((np.ones((data.obs.shape[0],1)), data.obs.nCount_RNA, encoded_array))
print(x[0:3,:])
print(x.shape)

In [ ]:
#### Design matrix : Intercept + Depth + cell-types + strain
x = np.column_stack((np.ones((data.obs.shape[0],1)), data.obs.nCount_RNA, encoded_array + y_strain))
print(x[0:3,:])
print(x.shape)

In [35]:
#### Design matrix : Intercept + Depth + cell-types + sample
x = np.column_stack((np.ones((data.obs.shape[0],1)), data.obs.nCount_RNA, encoded_array , y_DA_01, y_DA_02, y_LEW_01, y_LEW_02))
print(x[0:3,:])
print(x.shape)

[[1.000e+00 3.007e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00]
 [1.000e+00 2.176e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00]
 [1.000e+00 1.978e+03 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00]]
(23036, 23)


In [ ]:
#### Design matrix : Intercept + Depth + cell-types + sample + strain
### including sample as well as cluster as covariates
x = np.column_stack((np.ones((data.obs.shape[0],1)), data.obs.nCount_RNA, encoded_array, y_DA_01, y_DA_02, y_LEW_01, y_LEW_02, y_strain))
print(x[0:3,:])
print(x.shape)

In [36]:
#### Interaction terms
data = sm.datasets.scotland.load(as_pandas=True)
df = data.data

gamma_model = smf.glm(formula='YES ~ COUTAX + UNEMPF + COUTAX:UNEMPF + MOR',
data=df, family=sm.families.Gamma())


(23036, 23)
(23036, 5000)


In [8]:
### fit a poisson regression model to each gene and save the results

num_vars = x.shape[1]

### make an empty array to store the p-values and coefficients
pvalue = []
coefficient = []
yhat = []
tvalues = []
resid_pearson = []
resid_deviance = []
resid_response = []
resid_working = []
fittedvalues = []
nobs = []
models = []

pearson_chi2 = []
deviance = []
null_deviance = []

### time the fitting process
start_time = time.time()

for i in range(len(y[0])):
    y_a_gene = y[:, i]
    model = sm.GLM(y_a_gene, x, family=sm.families.Poisson())
    result = model.fit()
    #print(result.summary())
    
    #models.append([result])
    #coefficient.append([result.params])
    #pvalue.append([result.pvalues]) ## yhat == fittedvalue == mu
    #yhat.append([result.predict()])
    #fittedvalues.append([result.fittedvalues])

    #nobs.append([result.nobs])
    #tvalues.append([result.tvalues])
    resid_pearson.append([result.resid_pearson])
    resid_deviance.append([result.resid_deviance])
    resid_response.append([result.resid_response])
    #resid_working.append([result.resid_working])
    
    #pearson_chi2.append([result.pearson_chi2])
    #deviance.append([result.deviance])
    #null_deviance.append([result.null_deviance])

end_time = time.time()
print('time to fit the model: ', end_time - start_time)


time to fit the model:  204.0016176700592


In [ ]:

pvalue = np.asarray(pvalue).reshape(num_genes, num_vars)
coefficient = np.asarray(coefficient).reshape(num_genes, num_vars)
tvalues = np.asarray(tvalues).reshape(num_genes, num_vars)
#### print the head of the pvalues
print(pvalue[0:5, :])

yhat = np.asarray(yhat).reshape(num_genes, num_cells)
fittedvalues = np.asarray(fittedvalues).reshape(num_genes, num_cells)
resid_pearson = np.asarray(resid_pearson).reshape(num_genes, num_cells)
resid_deviance = np.asarray(resid_deviance).reshape(num_genes, num_cells)
resid_response = np.asarray(resid_response).reshape(num_genes, num_cells)
resid_working = np.asarray(resid_working).reshape(num_genes, num_cells)
nobs = np.asarray(nobs).reshape(num_genes, 1)

pearson_chi2 = np.asarray(pearson_chi2).reshape(num_genes, 1)
deviance = np.asarray(deviance).reshape(num_genes, 1)
null_deviance = np.asarray(null_deviance).reshape(num_genes, 1)


In [9]:
resid_pearson = np.asarray(resid_pearson).reshape(num_genes, num_cells)

In [ ]:
#### chosing a random gene to plot
i = 800 ## gene index
result = models[i][0] 
fittedvalues_i = fittedvalues[i]
y_i = y[:, i]

### get the summary of the model
print(result.summary())

In [ ]:
### diagnostic plots of the model outputs
plt.rcParams.update({'font.size': 10})

fig, ax = plt.subplots()
ax.scatter(fittedvalues_i, y[:, i])
line_fit = sm.OLS(y_i, sm.add_constant(fittedvalues_i, prepend=True)).fit()
abline_plot(model_results=line_fit, ax=ax, alpha=0.5)

ax.set_title('Model Fit Plot')
ax.set_ylabel('Observed values')
ax.set_xlabel('Fitted values')

sns.jointplot(fittedvalues_i, y[:, i], kind='scatter', stat_func=None, color='b', height=4)
sns.set_context(font_scale=0.9)                                                  
plt.title('Model Fit Plot')
plt.xlabel('Fitted values')
plt.ylabel('Observed values')
## change the size of the text in plot
plt.rc('font', size=10)

fig, ax = plt.subplots()
resid_pearson_i = resid_pearson[i]
ax.scatter(fittedvalues_i, resid_pearson_i, alpha=0.1)
#ax.hlines(0, 0, 7)
#ax.set_xlim(0, 7)
ax.set_title('Residual Dependence Plot')
ax.set_ylabel('Pearson Residuals')
ax.set_xlabel('Fitted values')

sns.jointplot(fittedvalues_i, resid_pearson_i, kind='scatter', stat_func=None, color='b', height=4)
sns.set_context(font_scale=0.9)                                                  
plt.title('Model Fit Plot')
plt.xlabel('Fitted values')
plt.ylabel('Pearson Residuals')
## change the size of the text in plot
plt.rc('font', size=10)
plt.show()

from scipy import stats
fig, ax = plt.subplots()
resid = resid_deviance[i]
resid_std = stats.zscore(resid)
ax.hist(resid_std, bins=25)
ax.set_title('Histogram of standardized deviance residuals')

from statsmodels import graphics
graphics.gofplots.qqplot(resid, line='r')

In [ ]:
#### calculate the goodness of fit of GLM model
# https://www.statsmodels.org/stable/generated/statsmodels.genmod.generalized_linear_model.GLMResults.html#statsmodels.genmod.generalized_linear_model.GLMResults

### statsmodels GLM source code:
# https://github.com/statsmodels/statsmodels/blob/main/statsmodels/genmod/families/links.py

### poisson family link function
# https://github.com/statsmodels/statsmodels/blob/main/statsmodels/genmod/families/family.py


print('log-likelihood of the model: ', result.llf)

### calculate the null deviance manually - float
# NullDeviance = 2*(LL(SaturatedModel)−LL(NullModel))
# The value of the deviance function for the model fit with a constant as the only regressor
# GLMResults.llf: Value of the loglikelihood function evalued at params. See statsmodels.families.family for distribution-specific loglikelihoods.
null_deivance_i = 2*(result.llnull - result.llf)
print('null deviance manual calculation: ', null_deivance_i) ## why the result is different from the result.null_deviance?
print('null deviance: ',result.null_deviance)


### calculate the deviance - float
# The value of the deviance function for the model fit with the full set of regressors
# ResidualDeviance = 2*(LL(SaturatedModel)−LL(ProposedModel))
deviance_i =result.deviance 
print('deviance: ', deviance_i)


### Response residual - array
# The response residuals are defined as endog - fittedvalues
response_residual_i = y_i - fittedvalues_i
print('response residual manual: ', response_residual_i[:3])
print('response residual is: ', resid_response[i][:3])

### Deviance residual - how is this calculated? - array
print('residual deviance is: ', resid_deviance[i][1:3])

  
## Working residual - response residuals scaled by the derivative of the inverse of the link function
# The working residuals are defined as resid_response/link’(mu)
# mu: The inverse of the link function at the linear predicted values. - then why is mu=yhat instead of mu=e^yhat? 
# https://www.statsmodels.org/dev/glm.html#link-functions
resid_working_i = resid_response[i]/result.family.link.deriv(result.mu) ## derivatives of the link function
print('working residual manual: ', resid_working_i[:3])
print('working residual is: ', resid_working[i][:3])


### The Pearson residuals - response residuals scaled by the square root of the variance function
# (endog - mu)/sqrt(VAR(mu)) where VAR is the distribution specific variance function
resid_pearson_i = (y_i - result.mu)/np.sqrt(result.family.variance(result.mu)) ## variance function of the poisson family is power?
print('pearson residual manual: ', resid_pearson_i[:3])
print('pearson residual is: ', resid_pearson[i][:3])


### calculate the Pearson’s Chi-Squared statistic 
# the sum of the squares of the Pearson residuals.
Pearson_Chi_Squared_i = sum(resid_pearson[i]**2)
print('Pearson_Chi_Squared manual: ', Pearson_Chi_Squared_i)
print('Pearson_Chi_Squared: ', result.pearson_chi2)


### calculate the BIC - float
# BIC : deviance - df_resid * log(nobs)
BIC_i = deviance_i - result.df_resid * np.log(nobs[0])
print('BIC manual: ', BIC_i)
print('BIC is: ', result.bic)


### calculate the AIC - float
# AIC: aike Information Criterion
# -2 * llf + 2 * (df_model + 1)
## df_model: rank of the regression matrix excluding the intercept:  df_model = k_exog - 1 = 0 is only strain is included
AIC_i = deviance_i + 2 * (result.df_model + 1)
AIC_i = -2 * result.llf + 2 * (result.df_model + 1)
print('AIC manual: ', AIC_i)
print('AIC is: ', result.aic)


### calculate the pseudo R-squared - float
pseudo_rsquared_i = 1 - abs(result.deviance/result.null_deviance)
print('pseudo R-squared manual: ', pseudo_rsquared_i)
#print('pseudo R-squared is: ', result.pseudo_rsquared)

### calculate the Wald test - float
# result.wald_test('x1 = x2 = 0')

# GLMResults.wald_test : Compute a Wald-test for a joint linear hypothesis.
# GLMResults.compare_lr_test : Likelihood ratio test for comparing two models.\n"

print(resid_response.T.shape)
print(y.shape)

In [38]:
def plot_pca(pca_scores, pca, num_components_to_plot, title='PCA of the data matrix'):
    
    for i in range(1, num_components_to_plot):
        ## color PCA based on strain
        plt.figure()
        plt.scatter(pca_scores[:,0], pca_scores[:,i], c=strain_color, s=1) 
        plt.xlabel('PC1')
        plt.ylabel('PC'+str(i+1))
        plt.title(title)
        plt.legend(handles=[mpatches.Patch(color='springgreen', label='LEW'),
                            mpatches.Patch(color='hotpink', label='DA')])
        plt.show()

        ## color PCA based on cluster
        plt.figure()
        plt.scatter(pca_scores[:,0], pca_scores[:,i], c=c, s=1)
        plt.xlabel('PC1')
        plt.ylabel('PC'+str(i+1))
        plt.title(title)
        plt.show()

        ###  ## color PCA based on sample
        plt.figure()
        plt.scatter(pca_scores[:,0], pca_scores[:,i], c=sample_color, s=1)
        plt.xlabel('PC1')
        plt.ylabel('PC'+str(i+1))
        plt.legend(handles=[mpatches.Patch(color='palegreen', label='LEW_01'),
                            mpatches.Patch(color='forestgreen', label='LEW_02'),
                            mpatches.Patch(color='pink', label='DA_01'),
                            mpatches.Patch(color='orchid', label='DA_02')], loc='best')
        plt.show()


    # plot the variance explained by each PC
    plt.figure()
    plt.plot(pca.explained_variance_ratio_)
    plt.title('Variance explained by each PC')
    plt.xlabel('PC')
    plt.ylabel('Variance explained')
    plt.show()


def plot_umap(pca_scores, pca , title='UMAP of the PC components of the gene expression matrix'):

    ### apply UMAP to teh PCA components
    reducer = umap.UMAP()
    embedding = reducer.fit_transform(pca_scores)
    print('embedding shape: ', embedding.shape)
    
    ### plot the UMAP embedding
    plt.figure()
    plt.scatter(embedding[:, 0], embedding[:, 1], c=strain_color, s=1)
    plt.title(title)
    plt.legend(handles=[mpatches.Patch(color='springgreen', label='LEW'),
                        mpatches.Patch(color='hotpink', label='DA')])
    plt.show()

    plt.figure()
    plt.scatter(embedding[:, 0], embedding[:, 1], c=sample_color, s=1)
    plt.title(title)
    plt.legend(handles=[mpatches.Patch(color='palegreen', label='LEW_01'),
            mpatches.Patch(color='forestgreen', label='LEW_02'),
            mpatches.Patch(color='pink', label='DA_01'),
            mpatches.Patch(color='orchid', label='DA_02')], loc='best')
    plt.show()

    plt.figure()
    plt.scatter(embedding[:, 0], embedding[:, 1], c=c, s=1)
    plt.title(title)
    plt.show()



#### generating the list of colors for clusters
colors = plt.cm.rainbow(np.linspace(0, 1, len(set(y_cluster)))) ### set the color for each cluster
my_color = {}
### define a dictionary to map the cluster number to the color
list_levels = list(set(y_cluster))
for i in range(len(set(list_levels))):
    my_color[list_levels[i]] = colors[i]
### generate a list containing the corresponding color for each cluster
c = [my_color[y_cluster[i]] for i in range(len(y_cluster))]


### generating the list of colors for samples
my_color = {'LEW_01': 'palegreen', 'LEW_02':'forestgreen', 'DA_01':'pink', 'DA_02':'orchid'}
### generate a list containing the corresponding color for each sample
sample_color = [my_color[y_sample[i]] for i in range(len(y_sample))]

my_color = {'LEW': 'springgreen', 'DA':'hotpink'}
strain_color = [my_color[y_strain_str[i]] for i in range(len(y_strain_str))]

### applying PCA to the data matrix
num_components = 30


In [ ]:
print(y.shape)
num_components = 30

In [ ]:
### using pipeline to scale the data first
pipeline = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=num_components))])
pca_scores = pipeline.fit_transform(y)
pca = pipeline.named_steps['pca']
pca_loading = pca.components_

plot_pca(pca_scores, pca, 5, title='PCA of data')
plot_umap(pca_scores, pca, title='UMAP of the PC components of the data')

In [ ]:
##### First implementation of Varimax rotation ####
## method: scale(original pc scores) %*% rotmat

## apply varimax rotation to the loadings
varimax_rot = varimax(pca_loading.T)
loadings_rot = varimax_rot['rotloadings']
rotmat = varimax_rot['rotmat']
pca_scores_rot = dot(pca_scores, rotmat)

print('loadings_rot shape: ', loadings_rot.shape)
print('rotmat shape: ', rotmat.shape)
print('pca_scores shape: ', pca_scores.shape)
print('pca_scores_rot shape: ', pca_scores_rot.shape)


plot_pca(pca_scores_rot, pca, 10, title='PCA of varimax rotated pearson residuals')
plot_umap(pca_scores_rot, pca, title='UMAP of the varimax-PCs on pearson residuals')



In [ ]:
##### Second implementation of Varimax rotation ####
# method: scale(original data) %*% rotatedLoadings

## apply varimax rotation to the loadings
#loadings_rot shape:  (2000, 10)
y_std = StandardScaler().fit_transform(y)
pca_scores_rot_2 = dot(y_std, loadings_rot)

### plot the rotated scores pca_scores_rot
plot_pca(pca_scores_rot_2, pca, 10, title='PCA of response residuals after varimax rotation')


In [ ]:
## visualize the loadings
plt.figure()
plt.imshow(loadings_rot, cmap="RdBu_r", vmin=-1, vmax=1)
plt.colorbar().ax.set_ylabel("$\\beta$", rotation=0)
plt.xticks([0, 1], ["PC1", "PC2"])
plt.title("PCA loadings (varimax rotated)")
plt.tight_layout()
plt.show()

In [ ]:
### applying PCA to the response residuals
pipeline = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=num_components))])
pca_scores = pipeline.fit_transform(resid_response.T)
pca = pipeline.named_steps['pca']
pca_loading = pca.components_

#print('explained variance ratio: ', pca.explained_variance_ratio_)
#print('singular values: ', pca.singular_values_)#
print('noise_variance: ', pca.noise_variance_)

plot_pca(pca_scores, pca, 2, title='PCA of response residuals')
plot_umap(pca_scores, pca, title='UMAP of the PC components of the response residuals')

In [41]:
### applying PCA to the pearson residuals

pipeline = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=num_components))])
pca_scores = pipeline.fit_transform(resid_pearson.T)
pca = pipeline.named_steps['pca']
pca_loading = pca.components_

#print('explained variance ratio: ', pca.explained_variance_ratio_)
#print('singular values: ', pca.singular_values_)#
print('noise_variance: ', pca.noise_variance_)

plot_pca(pca_scores, pca, 15, title='PCA of pearson residuals')
plot_umap(pca_scores, pca, title='UMAP of the PC components of the pearson residuals')


noise_variance:  0.9824045621837092


In [ ]:
##### First implementation of Varimax rotation ####
## method: scale(original pc scores) %*% rotmat

## apply varimax rotation to the loadings
varimax_rot = varimax(pca_loading.T)
loadings_rot = varimax_rot['rotloadings']
rotmat = varimax_rot['rotmat']
pca_scores_rot = dot(pca_scores, rotmat)

print('loadings_rot shape: ', loadings_rot.shape)
print('rotmat shape: ', rotmat.shape)
print('pca_scores shape: ', pca_scores.shape)
print('pca_scores_rot shape: ', pca_scores_rot.shape)


### plot the rotated scores pca_scores_rot
plot_pca(pca_scores_rot, pca, 30, title='PCA of pearson residuals after varimax rotation')
plot_umap(pca_scores, pca, title='UMAP of varimax-PCA on pearson residuals')



In [ ]:
### get the UMAP cooridates for each data point from the embedding
### get the head of the embedding
embedding[:len(y_sample), :]
### add cluster information to the embedding
embedding = np.concatenate((embedding, y_cluster_int.reshape(-1, 1)), axis=1)
### print head of the embedding
print(embedding[:len(y_sample), :])

x_coord = 12.5
y_coord = 2
### subset embedding to get the coordinates od data bellow  -4 in y axis and below 0 in x axis
embedding_sub = embedding[(embedding[:, 1] < y_coord) & (embedding[:, 0] > x_coord), :]

## get a summary of cluster values in the last column of the embedding
print(pd.Series(embedding_sub[:, -1]).value_counts())

In [ ]:
### applying PCA to the deviance residuals
pipeline = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=num_components))])
pipeline.fit_transform(resid_deviance)
pca = pipeline.named_steps['pca']

#pca = PCA(n_components=20)
#pca.fit(resid_response)

#print('explained variance ratio: ', pca.explained_variance_ratio_)
#print('singular values: ', pca.singular_values_)#
print('noise_variance: ', pca.noise_variance_)

plot_pca(pca, 4, title='PCA of deviance residuals')
plot_umap(pca, title='UMAP of the PC components of the deviance residuals')

In [ ]:
### convert the pvalue matrix to a pandas dataframe
pvalue_pd = pd.DataFrame(pvalue)

### check the colnames of data matrix
gene_names = data.var_names[gene_sums != 0]
gene_names = pd.DataFrame(gene_names[gene_idx])

### concatenate the gene names with the pvalues
gene_pval_df = pd.concat([gene_names, pvalue_pd], axis=1)
column_names = ['gene', 'intercept','strain_DA', 'DA_01', 'LEW_01', 'libsize']
# add column_names to the gene_pval_df dataframe
gene_pval_df.columns = column_names
gene_pval_df.head()
### sort the dataframe by strain_DA pvalue
gene_pval_df.sort_values(by=['intercept'], inplace=True)
gene_pval_df.head(20)

In [ ]:
#### con
### convert the coefficient matrix to a pandas dataframe
coefficient_pd = pd.DataFrame(coefficient)


### concatenate the gene names with the pvalues
gene_coeff_df = pd.concat([gene_names, coefficient_pd], axis=1)
column_names = ['gene', 'intercept','strain_DA', 'DA_01', 'LEW_01']
# add column_names to the gene_pval_df dataframe
gene_coeff_df.columns = column_names
gene_coeff_df.head()

### sort the dataframe by strain_DA coefficient
gene_coeff_df.sort_values(by=['DA_01'], inplace=True)
gene_coeff_df.head(20)

### show the row containing the gene name Itgal
gene_name = 'Itgal'
gene_coeff_df.loc[gene_coeff_df['gene'] == gene_name]